### Performance across sessions binned
- Calculate performance across sessions and bin sessions into roughly 10 sessions per bin. This one also calculates "struc_in_unstruc" sessions

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.b2a.filter_by_trials(min_trials=100, clip_max=150)
    perf = mab.get_optimal_choice_probability(bin_size=10)

    df = pd.DataFrame(data=perf, columns=np.arange(perf.shape[1]) + 1)
    df["session_bin"] = np.arange(perf.shape[0]) + 1
    df = df.melt(id_vars="session_bin", var_name="trial_id", value_name="perf")
    df.sort_values(by=["session_bin", "trial_id"], inplace=True)

    df["name"] = exp.sub_name
    df["grp"] = "struc" if mab.is_structured else "unstruc"

    perf_df.append(df)

    if ~mab.is_structured:
        mab2 = exp.b2a.filter_by_trials(min_trials=100, clip_max=150)
        session_prob_sum = mab2.probs[mab2.is_session_start.astype(bool)].sum(axis=1)
        good_sessions = mab2.sessions[session_prob_sum == 1]
        assert np.all(good_sessions)
        mab2 = mab2.filter_by_session_id(good_sessions)

        perf = mab2.get_optimal_choice_probability(bin_size=10)

        df2 = pd.DataFrame(data=perf, columns=np.arange(perf.shape[1]) + 1)
        df2["session_bin"] = np.arange(perf.shape[0]) + 1
        df2 = df2.melt(id_vars="session_bin", var_name="trial_id", value_name="perf")
        df2.sort_values(by=["session_bin", "trial_id"], inplace=True)

        df2["name"] = exp.sub_name
        df2["grp"] = "struc_in_unstruc"
        perf_df.append(df2)

perf_df = pd.concat(perf_df, ignore_index=True)
mab_subjects.GroupData().save(perf_df, "perf_100min150max_10bin")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.perf_100min150max_10bin
df1 = df[((df["session_bin"] > 30) & (df["grp"] == "unstruc"))]
df1["grp"] = "unstruc_late"
df = pd.concat([df, df1], ignore_index=True)

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)
hue_order = ["unstruc", "struc", "struc_in_unstruc", "unstruc_late"]
plot_kw = dict(data=df, x="trial_id", y="perf", hue="grp", hue_order=hue_order, ax=ax)
sns.lineplot(
    palette=["#f77189", "#36ada4", "#a48cf4", "#f7c94a"],
    # edgecolor="white",
    # facecolor=(0, 0, 0, 0),
    # alpha=0.4,
    err_kws=dict(edgecolor="none"),
    errorbar="se",
    **plot_kw,
)


# orders = ["unstruc", "struc"]
# pairs = [(("unstruc"), ("struc"))]
# annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
# annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()
# ax.grid(True)
ax.set_title("Performance")
ax.set_ylabel("Performance")
ax.set_ylim(0.45, 1)
ax.set_xticks([1, 50, 100, 150])
# ax.legend("")

### Performance in all, easy, and hard sessions
- Comparing Unstruc and Struc where the difference in probability is more than 0.4 

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess
trial_filter = dict(min_trials=100, clip_max=100)

perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(**trial_filter).filter_by_deltaprob(
        delta_min=0.05, delta_max=None
    )
    mab_easy = task.filter_by_deltaprob(delta_min=0.4)
    mab_hard = task.filter_by_deltaprob(delta_min=0.1, delta_max=0.35)

    perf = task.get_optimal_choice_probability()
    perf_easy = mab_easy.get_optimal_choice_probability()
    perf_hard = mab_hard.get_optimal_choice_probability()

    df = pd.DataFrame(
        dict(
            trial_id=np.arange(perf.size) + 1,
            perf=perf,
            perf_easy=perf_easy,
            perf_hard=perf_hard,
            name=exp.sub_name,
            grp="struc" if task.is_structured else "unstruc",
            first_experience=True if "Exp1" in exp.sub_name else False,
        )
    )
    perf_df.append(df)

perf_df = pd.concat(perf_df, ignore_index=True)
mab_subjects.GroupData().save(perf_df, "perf_difficulty_level")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from mab_colors import colors_2arm
from statplotannot.plots import SeabornPlotter, fix_legend

fig = plotting.Fig(5, 3, num=1, fontsize=10)

grpdata = mab_subjects.GroupData()
df = grpdata.perf_difficulty_level
df = df[df["first_experience"] == True]

# ax = fig.subplot(fig.gs[0])
hue_order = ["unstruc", "struc"]
linestyles = ["-", "--", ":"]
titles = [
    "All permutations",
    "Easy\n(DeltaP>=40)",
    "Hard\n(DeltaP<=30)",
]

for i, y in enumerate(["perf", "perf_easy", "perf_hard"]):

    ax = fig.subplot(fig.gs[i])
    plot_kw = dict(data=df, x="trial_id", y=y, hue="grp", hue_order=hue_order, ax=ax)
    sns.lineplot(
        palette=colors_2arm(),
        lw=1.1,
        # ls=linestyles[i],
        ls="-",
        err_kws=dict(edgecolor="none"),
        errorbar="se",
        **plot_kw,
    )
    ax.set_ylim(0.45, 1)
    fix_legend(ax, only_labels=True, fw="bold", fs=10)
    ax.grid(axis="y", zorder=-1, alpha=0.5)
    ax.set_ylabel("Pr(Optimal Choice)")
    ax.set_xticks([1, 25, 50, 75, 100])
    ax.set_title(titles[i])

# ax.legend("")

### Performance matrix
- Bins are represented by probability combination

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

prob_perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100)

    probs = np.unique(task.probs[task.is_session_start.astype(bool)], axis=0)
    print(probs)

    arr = []
    for prob in probs:
        if prob[0] != prob[1]:
            prob_new = np.array([prob, prob[::-1]])
            prob2 = np.array([prob[::-1], prob])
        else:
            prob_new = np.array([prob])
        prob_perf = task.filter_by_probs(prob_new).get_optimal_choice_probability()
        delta_improvement = np.mean(prob_perf[-5:]) - np.mean(prob_perf[:5])
        final_perf = np.mean(prob_perf[-5:])
        # middle_perf = np.mean(prob_perf[50:60])
        # prob_perf = gaussian_filter1d(prob_perf, sigma=2)
        arr.append([prob[0], prob[1], final_perf, delta_improvement])
        arr.append([prob[1], prob[0], final_perf, delta_improvement])

    df = pd.DataFrame(
        np.array(arr),
        columns=["prob1", "prob2", "final_perf", "delta_improvement"],
    )
    df["name"] = exp.sub_name
    df["grp"] = "struc" if task.is_structured else "unstruc"
    df["first_experience"] = True if "Exp1" in exp.sub_name else False

    prob_perf_df.append(df)

prob_perf_df = pd.concat(prob_perf_df, ignore_index=True)
mab_subjects.GroupData().save(prob_perf_df, "perf_probability_matrix")

In [ ]:
import matplotlib.pyplot as plt
from neuropy.plotting import Fig
import seaborn as sns
from scipy.stats import binned_statistic_2d
from mab_colors import colors_2arm
from statplotannot.plots import fix_legend
import mab_subjects
import numpy as np

fig = Fig(5, 3, size=(8.5, 11), num=1, fontsize=10)

mat_df = mab_subjects.GroupData().perf_probability_matrix
mat_df = mat_df[mat_df["first_experience"] == True]
equal_prob_bool = mat_df["prob1"] == mat_df["prob2"]
mat_df = mat_df[~equal_prob_bool]

vmin = [0.4, 0.1]
vmax = [0.9, 0.4]

for c, col_name in enumerate(["final_perf", "delta_improvement"]):

    for g, grp in enumerate(["unstruc", "struc"]):
        df = mat_df[mat_df["grp"] == grp]
        x, y, values = df["prob1"], df["prob2"], df[col_name]
        bins = np.linspace(0, 0.9, 10) + 0.05
        centers = (bins[:-1] + bins[1:]) / 2
        centers = np.round(centers, 2)

        H, xedges, yedges, _ = binned_statistic_2d(
            x, y, values, statistic=np.mean, bins=bins
        )

        ax = fig.subplot(fig.gs[c, g])
        cplot = ax.pcolormesh(
            xedges,
            yedges,
            H.T,
            cmap="plasma",
            vmin=vmin[c],
            vmax=vmax[c],
            shading="auto",
        )
        ax.set_xticks(centers[::2])
        ax.set_yticks(centers[::2])
        ax.set_xlabel("Arm 1 prob.")
        ax.set_ylabel("Arm 2 prob.")
        ax.set_title(f"{grp}: {col_name.replace('_', ' ').title()}")
        plt.colorbar(cplot, ax=ax, label="Pr(High)", shrink=0.6, anchor=(0, 0.9))

    df_correlated = mat_df[(mat_df["prob1"] + mat_df["prob2"]) == 1]

    ax = fig.subplot(fig.gs[c, 2])
    plot_kw = dict(
        data=df_correlated,
        x="prob1",
        y=col_name,
        hue="grp",
        ax=ax,
        palette=colors_2arm(),
    )
    # sns.stripplot(**plot_kw, dodge=True, size=3, alpha=0.5)
    sns.pointplot(
        **plot_kw,
        dodge=True,
        # markers=["o", "s"],
        # linestyles=["-", "--"],
        lw=2,
        errorbar="se",
        alpha=0.6,
    )

    if c == 0:
        ax.set_ylim(0.1, 1.2)
    if c == 1:
        ax.set_ylim(-0.4, 0.9)
    ax.set_xlabel("Arm 1 - Arm 2 prob.")
    xticks = np.delete(np.arange(0.1, 1, 0.1).round(1), 4)

    ax.set_xticks(np.arange(8), [f"{p1}-{p2}" for p1, p2 in zip(xticks, xticks[::-1])])
    ax.tick_params(axis="x", rotation=45)
    ax.set_title("Performance improvement\n1st to last 5 trials")
    ax.set_ylabel("Delta performance")
    fix_legend(ax, frameon=True)
    # ax.legend_.remove()

### Entropy of choices in equal probability arms

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d

exps = mab_subjects.unstruc.allsess

entropy_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100).filter_by_deltaprob(
        delta_min=0, delta_max=0
    )
    entropy = task.get_choice_entropy()

    df = pd.DataFrame(
        {
            "trial_id": np.arange(len(entropy)) + 1,
            "entropy": entropy,
            "name": exp.sub_name,
            "grp": "struc" if task.is_structured else "unstruc",
            "first_experience": True if "Exp1" in exp.sub_name else False,
        }
    )

    entropy_df.append(df)

entropy_df = pd.concat(entropy_df, ignore_index=True)
mab_subjects.GroupData().save(entropy_df, "entropy_equal_probs")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import mab_subjects
from neuropy import plotting
from mab_colors import colors_2arm

df = mab_subjects.GroupData().entropy_equal_probs
df = df[df["first_experience"] == True]

fig = plotting.Fig(1, 3, size=(11, 3), num=1)
ax = fig.subplot(fig.gs[0])
sns.lineplot(data=df, x="trial_id", y="entropy", color=colors_2arm()[0], errorbar="se")
ax.set_title("Entropy of choosing between equal probability arms")
ax.set_ylim(0, 1)
ax.set_ylabel("Entropy")
# ax.legend_.remove()

### Impure environments individual metrics

In [2]:
import matplotlib.pyplot as plt
from banditpy.plots import plot_trial_by_trial_2Arm
from neuropy import plotting
from scipy import stats
import mab_subjects
import numpy as np


exps = mab_subjects.mostly_unstruc.allsess + mab_subjects.mostly_struc.allsess

fig = plotting.Fig(7, 6)

for i, exp in enumerate(exps):

    task = exp.b2a
    probs_corr = task.probs_corr

    if probs_corr < -0.5:
        grp = "struc"
    else:
        grp = "unstruc"

    H, xedges, yedges = task.get_prob_hist_2d(stat="prop")
    # choices[choices == 1] = 2
    # choices[task.choices == 2] = 1
    # task.choices = choices

    perf = task.filter_by_trials(
        min_trials=100, clip_max=100
    ).get_optimal_choice_probability()

    ax = fig.subplot(fig.gs[3 * i])
    ax.plot(perf, color="k")
    # task.plot_trial_by_trial(ax=ax)
    ax.set_ylim(0.4, 0.9)
    ax.set_xlabel("Trial_id")
    ax.set_ylabel("Choice (High)")
    ax.set_title(f"{exp.sub_name}'s performance")

    ax2 = fig.subplot(fig.gs[3 * i + 1])
    cplot = ax2.pcolormesh(
        xedges,
        yedges,
        H.T,
        cmap="hot",
        vmin=0,
        # vmax=14,
        shading="auto",
    )
    cb = plt.colorbar(cplot, ax=ax2, shrink=0.5)
    # ax2.imshow(prob_mat)
    ax2.set_xlabel("Prob1")
    ax2.set_ylabel("Prob2")
    ax2.set_title(f"{exp.sub_name}'s probability combinations")
    # ax2.set_xticks([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
    # ax2.set_yticks([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
    cb.set_label("Counts")

    ax3 = fig.subplot(fig.gs[3 * i + 2])
    h, bins = task.get_trials_hist()

    ax3.plot(bins, h, color="k")
    ax3.set_xlim(0, 300)

###  Impure: Overall performance
- Choosing better port with and without impure combinations

In [3]:
import matplotlib.pyplot as plt
from banditpy.plots import plot_trial_by_trial_2Arm
from neuropy import plotting
from scipy import stats
import mab_subjects
import numpy as np
import pandas as pd


exps = mab_subjects.mostly_unstruc.good_sess + mab_subjects.mostly_struc.good_sess

perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)

    task = exp.b2a
    probs_corr = task.probs_corr
    print(probs_corr)

    task_filt = task.filter_by_trials(min_trials=100, clip_max=100)
    perf_overall = task_filt.get_optimal_choice_probability()
    probs_all = task_filt.probs

    df = pd.DataFrame()
    df["trial_id"] = np.arange(perf_overall.shape[0]) + 1
    df["overall"] = perf_overall

    if probs_corr < -0.5:
        grp = "struc"
        dependent_mask = probs_all.sum(axis=1) == 1.0
        independent_mask = ~dependent_mask

        perf_pure = task_filt._filtered(dependent_mask).get_optimal_choice_probability()

        perf_impure = task_filt._filtered(
            independent_mask
        ).get_optimal_choice_probability()
    else:
        grp = "unstruc"
        dependent_mask = probs_all.sum(axis=1) == 1.0
        independent_mask = ~dependent_mask

        perf_impure = task_filt._filtered(
            dependent_mask
        ).get_optimal_choice_probability()

        perf_pure = task_filt._filtered(
            independent_mask
        ).get_optimal_choice_probability()

    df["pure"] = perf_pure
    df["impure"] = perf_impure
    df["name"] = exp.sub_name
    df["grp"] = grp
    perf_df.append(df)


perf_df = pd.concat(perf_df, ignore_index=True)
# mab_subjects.GroupData().save(perf_df, "perf_100min150max_10bin")

BGM0
-0.161009709830758
BGF1
-0.1622709392736449
BGF2
-0.2442685111991692
BGM1
-0.786882006006754
BGF0
-0.9606201653671316
BGM3
-0.6164415772340107
BGF4
-0.917934269206941


In [5]:
from statplotannot.plots import SeabornPlotter, fix_legend
from mab_colors import colors_2arm
import seaborn as sns
from neuropy import plotting

fig = plotting.Fig(3, 3, size=(8.5, 6), fontsize=10)

for i, y in enumerate(["overall", "pure", "impure"]):
    ax = fig.subplot(fig.gs[i])
    sns.lineplot(
        data=perf_df,
        x="trial_id",
        y=y,
        hue="grp",
        palette=colors_2arm(),
        errorbar="se",
    )
    fix_legend(ax)
    ax.set_ylim(0.1, 0.9)
    ax.set_ylabel("P(High)")
    ax.set_title(y)

    ax.grid(axis="y", zorder=-1, alpha=0.5)

Impure: performance grid